# test 데이터 하나가 잘 inference 되는지 확인하고 진행 (api는 신중히 접근)

### data reforming to using gpt
- api키는 무조건 자기 환경변수로 저장해두거나 지울것.


In [ ]:
import os

from openai import OpenAI
client = OpenAI()

print(os.environ.get("OPENAI_API_KEY"))

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)

print(completion.choices[0].message)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [1]:
import json
import os
import time
from openai import OpenAI
client = OpenAI()


# Set your OpenAI API key

def translate_text(text):
    if not text.strip():
        print("text의 내용이 비어있기 때문에 skip 됬습니다.")
        return ""  # Return empty string if note is empty
    
    prompt = f"Translate the following Korean text to English and look like expert:\n\n{text}"
    
    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        )
        translated_text = completion.choices[0].message
        return translated_text
    except:
        print("실패했습니다.")
        return ""


text = ""
output = translate_text(text)
print(output)

time.sleep(10)

text = "고주파수에서 앞뒤 1초의 강도가 매우 약함.\n저중주파수(2kHz, 3kHz)에서 0.5kHz 폭만큼 강도가 약해지는 구간 관측됨.\n스파이크 패턴이 비주기적으로 저주파수에서 고주파수까지 존재하나 높은 주파수일수록 강도가 점점 작아짐.\n고중주파수(6kHz-7kHz)에서 1kHz 폭만큼 강도가 약해지는 구간 관측됨."
output = translate_text(text)
print(output)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

# 밑의 코드는 실행하면 바로 api를 사용해서 data transform이 진행됨!!!!!!!!!

In [ ]:
import json
import os
import time
from openai import OpenAI
client = OpenAI()


def translate_text(text):
    if not text.strip():
        print("text의 내용이 비어있기 때문에 skip 됬습니다.")
        return ""  # Return empty string if note is empty
    
    prompt = f"Translate the following Korean text to English and look like expert:\n\n{text}"
    
    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        )
        translated_text = completion.choices[0].message
        return translated_text
    except:
        print("실패했습니다.")
        return ""

def translate_notes_in_json(input_file, output_file, max_entries=None):
    with open(input_file, 'r', encoding='utf-8') as infile:
        data = json.load(infile)
    
    audio_features = data.get("audio_features", [])
    
    # Filter out entries with empty 'note' fields
    non_empty_features = [feature for feature in audio_features if feature.get('note', '').strip()] # 빈 노트는 건너뜀
    
    translated_data = []

    for idx, feature in enumerate(non_empty_features):
        if max_entries and idx >= max_entries:
            break
        
        note = feature.get('note', '').strip()
        translated_note = translate_text(note)
        feature['note'] = translated_note  # Update the note with the translated text
        translated_data.append(feature)
        print(f"Translated note for file {feature.get('file_path', '')}")
    
    # Save the translated data into the output_file
    with open(output_file, 'w', encoding='utf-8') as outfile:
        json.dump({"audio_features": translated_data}, outfile, ensure_ascii=False, indent=4)
    
    print(f"Translated data has been saved to {output_file}")

if __name__ == "__main__":
    input_json = "audio_features_note.json"  # Input file with Korean notes
    output_json = "audio_features_note_transform.json"  # Output file with translated notes
    # Set max_entries to limit the number of entries processed during testing
    translate_notes_in_json(input_json, output_json, max_entries=None)


### GPT-4o Mini 토큰 비용 계산

#### **주어진 조건**
- **입력:** 200자 미만의 한국어 → 약 `100~120 토큰`
- **출력:** 400자 정도 되는 영어 → 약 `200~250 토큰`
- **데이터 개수:** 400개
- **GPT-4o Mini 토큰 비용:**
  - **입력 토큰:** 1백만 개당 **$0.15**
  - **출력 토큰:** 1백만 개당 **$0.60**

---

### **계산 과정**

1. **평균 입력/출력 토큰 계산**
   - **평균 입력 토큰:** \( \frac{100 + 120}{2} = 110 \)  
   - **평균 출력 토큰:** \( \frac{200 + 250}{2} = 225 \)  
   - **평균 총 토큰 (입력 + 출력):** \( 110 + 225 = 335 \)

2. **총 토큰 수**
   - \( 335 \, \text{토큰/데이터} \times 400 \, \text{데이터} = 134,000 \, \text{토큰} \)

3. **비용 계산**
   - **입력 토큰 비용:**  
     \( 134,000 \, \text{토큰 중 입력 토큰} \)은 대략 \( 110 \, \text{토큰} \)이므로,  
     \( 110 \times 400 = 44,000 \, \text{입력 토큰} \)  
     - 1백만 개당 $0.15:
     \[
     \frac{44,000}{1,000,000} \times 0.15 = 0.0066 \, \text{USD}
     \]
   
   - **출력 토큰 비용:**  
     \( 134,000 \, \text{토큰 중 출력 토큰} \)은 대략 \( 225 \, \text{토큰} \)이므로,  
     \( 225 \times 400 = 90,000 \, \text{출력 토큰} \)  
     - 1백만 개당 $0.60:
     \[
     \frac{90,000}{1,000,000} \times 0.60 = 0.054 \, \text{USD}
     \]

4. **총 비용**
   - **입력 비용 + 출력 비용**:
     \[
     0.0066 + 0.054 = 0.0606 \, \text{USD}
     \]

---

### **결과**
- **총 토큰 수:** 134,000 토큰
- **예상 비용:** 약 **$0.0606** (모델 요금이 $0.15/1M 입력 토큰, $0.60/1M 출력 토큰일 경우)

---

**참고**: 실제 비용은 사용량, 토큰 크기, 요금 정책에 따라 달라질 수 있습니다. OpenAI의 [최신 요금 정책](https://openai.com/api/pricing/?utm_source=chatgpt.com)을 확인하여 정확한 비용을 확인하세요.


In [ ]:
import json
import os
import shutil

# -----------------------------
# 설정 파라미터
# -----------------------------

# 원본 JSON 파일 경로
AUDIO_FEATURES_JSON = "../../extract_feature_code/audio_features.json"

# 노트 정보가 담긴 JSON 파일 경로
AUDIO_FEATURES_NOTE_JSON = "../../extract_feature_code/audio_features_note_transform.json"

# 백업을 위한 원본 파일 복사 경로
BACKUP_JSON = "../../extract_feature_code/audio_features_backup.json"

# -----------------------------
# 함수 정의
# -----------------------------

def backup_file(original_file, backup_file):
    """
    원본 파일을 백업합니다.
    """
    try:
        shutil.copyfile(original_file, backup_file)
        print(f"백업 성공: {backup_file}")
    except Exception as e:
        print(f"백업 실패: {e}")
        raise

def load_json(file_path):
    """
    JSON 파일을 로드합니다.
    """
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"JSON 파일을 찾을 수 없습니다: {file_path}")
    
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    return data

def update_notes(original_data, notes_data):
    """
    original_data의 각 항목에 notes_data의 note를 업데이트합니다.
    단, notes_data의 note가 빈 문자열("")인 경우 업데이트하지 않습니다.
    """
    # file_path를 키로 하는 딕셔너리 생성 (빠른 검색을 위해)
    notes_dict = { 
        os.path.normpath(entry['file_path']): entry['note'] 
        for entry in notes_data.get("audio_features", [])
    }
    
    updated_count = 0
    skipped_count = 0
    not_found = []
    
    for feature in original_data.get("audio_features", []):
        # 파일 경로 정규화 (운영체제에 따라 경로 구분자가 다를 수 있음)
        feature_path = os.path.normpath(feature.get("file_path", ""))
        
        if feature_path in notes_dict:
            new_note = notes_dict[feature_path]
            if new_note.strip():  # 새로운 노트가 빈 문자열이 아닌 경우에만 업데이트
                feature['note'] = new_note  # 노트 업데이트
                updated_count += 1
                # print(f"업데이트됨: {feature_path}")
            else:
                skipped_count += 1
                # print(f"건너뜀 (빈 노트): {feature_path}")
        else:
            not_found.append(feature_path)
    
    return updated_count, skipped_count, not_found

def save_json(data, file_path):
    """
    데이터를 JSON 파일로 저장합니다.
    """
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"업데이트된 JSON 파일이 저장되었습니다: {file_path}")

# -----------------------------
# 메인 실행 흐름
# -----------------------------

def main():
    # 1. 원본 파일 백업
    backup_file(AUDIO_FEATURES_JSON, BACKUP_JSON)
    
    # 2. JSON 파일 로드
    original_data = load_json(AUDIO_FEATURES_JSON)
    notes_data = load_json(AUDIO_FEATURES_NOTE_JSON)
    
    # 3. 노트 업데이트
    updated_count, skipped_count, not_found = update_notes(original_data, notes_data)
    
    # 4. 업데이트된 데이터 저장
    save_json(original_data, AUDIO_FEATURES_JSON)
    
    # 5. 결과 요약
    print(f"총 {updated_count}개의 항목이 업데이트되었습니다.")
    print(f"총 {skipped_count}개의 항목이 빈 노트로 인해 건너뛰어졌습니다.")
    if not_found:
        print(f"노트를 찾을 수 없는 파일 경로 ({len(not_found)}개):")
        for path in not_found:
            print(f" - {path}")

if __name__ == "__main__":
    main()
